# Capítulo 9 - Support Vector Machines

## Lab: SVM com múltiplas classes - Aplicação nos dados de expressão gênica

Esse exercício usa o dataset [Khan](http://bioinf.ucd.ie/people/aedin/R/full_datasets/).

In [19]:
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
import seaborn as sns

# Parâmetros estéticos dos gráficos:
set_palette = "Paired"
axes_style = "ticks"

linha = 50*"-"

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [38]:
df = pd.read_csv("../data/khan_train.csv", index_col=0)
column_names = df.columns

print(f"Tamanho do dataset: {df.shape}")

round(df.describe().T, 2)

Tamanho do dataset: (2308, 64)


,count,mean,std,min,25%,50%,75%,max
EWS.T1,2308.0,0.90,0.90,0.04,0.30,0.59,1.09,4.26
EWS.T2,2308.0,0.91,0.90,0.02,0.34,0.63,1.15,7.62
EWS.T3,2308.0,0.91,0.91,0.04,0.33,0.60,1.14,5.58
EWS.T4,2308.0,0.92,1.19,0.04,0.28,0.50,1.05,10.48
EWS.T6,2308.0,0.94,0.87,0.05,0.38,0.65,1.19,6.11
EWS.T7,2308.0,0.92,0.93,0.06,0.37,0.62,1.09,7.15
EWS.T9,2308.0,0.92,0.85,0.09,0.43,0.65,1.08,7.76
EWS.T11,2308.0,0.92,1.04,0.10,0.35,0.57,1.04,7.69
EWS.T12,2308.0,0.91,1.15,0.04,0.27,0.52,1.04,8.81
EWS.T13,2308.0,0.94,1.34,0.06,0.31,0.54,1.02,19.28


In [49]:
df_transpose = df.T.copy()
df_transpose = df_transpose.reset_index()
df_transpose["target"] = df_transpose["index"].str.split('.', expand=True)[0]
df_transpose.head()

,index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,2210,2211,2212,2213,2214,2215,2216,2217,2218,2219,2220,2221,2222,2223,2224,2225,2226,2227,2228,2229,2230,2231,2232,2233,2234,2235,2236,2237,2238,2239,2240,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250,2251,2252,2253,2254,2255,2256,2257,2258,2259,2260,2261,2262,2263,2264,2265,2266,2267,2268,2269,2270,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280,2281,2282,2283,2284,2285,2286,2287,2288,2289,2290,2291,2292,2293,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303,2304,2305,2306,2307,2308,target
0,EWS.T1,3.2025,0.0681,1.0460,0.1243,0.4941,3.1207,3.7106,1.8416,1.2607,2.9001,4.0270,1.0643,4.0651,1.4730,2.7932,0.4815,1.4482,3.3214,0.7022,1.7260,1.5136,3.9255,0.5296,3.9098,3.7136,4.0016,1.1819,2.1459,0.6662,1.7203,0.2690,1.4677,0.6380,2.1497,3.7382,2.7538,0.8587,0.9320,1.0597,0.2684,1.1909,3.5375,0.1115,0.8346,1.5854,1.6778,3.9576,3.7285,3.6545,0.3247,3.5717,0.8967,2.3148,0.3194,3.4188,0.8722,3.9192,0.1288,3.1675,3.3678,3.6157,3.8972,1.2217,3.3361,1.0708,0.3088,0.1087,0.7344,0.6572,0.1167,0.4576,0.5910,0.1106,0.3945,0.1479,0.9519,2.6286,0.5856,0.1956,0.6629,0.3520,0.2393,3.6149,1.5132,0.5016,0.0848,1.0004,0.2410,0.3863,0.5780,0.4000,0.8543,0.0766,3.3357,0.1747,0.5405,0.7659,0.1336,1.5044,...,0.3504,0.6603,0.3518,1.3459,0.7639,0.9915,0.1247,0.2713,0.9123,0.5793,0.5820,1.4847,1.0220,1.8572,0.3142,1.1105,0.3092,0.1204,1.3403,0.0877,2.0164,0.7759,0.6366,0.0836,2.6887,0.5168,0.3635,0.4155,0.5783,2.0922,2.7577,0.8504,0.3465,0.3391,0.3785,0.0727,0.3933,0.1424,0.3721,1.4411,0.5301,0.1951,0.2269,0.7207,0.2212,0.3403,2.8931,0.1466,0.5841,0.3708,3.1939,0.8607,0.2362,3.1181,0.3351,0.8439,0.2712,0.5255,0.1081,0.1258,1.6173,0.4661,0.7599,0.2745,0.9985,0.1059,0.5853,3.8000,0.1275,0.5428,0.9514,1.8408,0.2024,0.6766,0.5689,0.5604,0.4027,1.0055,0.4548,0.2620,0.3451,0.7974,0.8585,0.4701,0.2156,0.3792,0.4419,0.2128,0.7948,0.7653,1.6679,0.1493,0.6918,1.4151,0.2756,0.1521,0.3175,0.7240,0.2044,EWS
1,EWS.T2,1.6547,0.0710,1.0409,0.0520,0.2045,2.1609,2.4452,1.1473,0.7371,1.9989,2.6131,0.8541,4.7284,2.4784,1.5103,0.8961,0.4850,2.3431,0.2531,1.7841,1.0886,5.9544,0.5337,2.7007,3.2339,3.4448,2.8296,0.9659,0.4829,1.3528,0.7262,1.2882,2.5171,3.0538,2.8671,1.9144,0.5963,0.9130,0.5030,1.3195,0.9951,3.1554,0.1146,0.6188,0.5013,1.6110,3.0636,1.7920,1.3757,0.3092,3.7014,0.7999,2.1661,0.3089,2.7758,2.0965,5.4148,0.0782,3.7242,3.6836,3.3596,7.6184,1.5961,2.4137,0.9393,0.4445,0.1556,0.6798,0.5171,0.1669,0.5398,0.4766,0.1132,0.2986,0.3997,1.6803,2.3320,0.4157,0.2604,0.5328,0.2067,0.2649,2.7040,1.2733,0.3463,0.1798,1.2508,0.2186,0.6392,0.1695,0.2286,0.4962,0.0543,1.3897,0.2190,0.7007,0.8402,0.2142,1.0901,...,0.4315,0.2567,0.9103,2.8039,0.3869,1.9895,0.1600,0.6718,1.1728,0.6937,0.6032,1.4177,1.5295,0.5944,0.2930,0.5310,0.0566,0.1017,0.1431,0.0391,0.6081,0.4266,0.3835,0.0818,1.8431,0.3684,0.2008,0.2120,0.7393,2.4250,1.4471,0.8495,0.9536,0.4104,0.5541,0.1485,0.4103,0.1395,0.4905,1.0295,0.7580,0.6851,1.5571,0.6841,0.6693,0.6007,1.1979,0.1994,0.8453,1.0842,2.1466,1.1005,0.3896,1.7622,0.7170,0.6113,0.3613,0.5445,0.7437,0.1938,1.0460,0.3609,0.5791,1.1683,1.2184,0.1118,0.6409,1.8589,0.1370,0.6127,0.5862,1.3508,0.5225,0.7216,0.6697,0.4791,0.2255,0.6509,0.2902,0.2171,1.6517,1.4018,1.9379,0.3307,0.1734,2.2825,0.3584,0.4693,1.4629,1.0665,3.6014,0.3048,1.7957,1.0701,0.2688,0.1932,0.4140,1.2708,0.2990,EWS
2,EWS.T3,3.2779,0.1160,0.8926,0.1014,0.2818,1.9773,3.2590,1.4106,0.9548,2.0775,4.8139,0.4257,4.7120,2.7548,1.9162,1.2710,1.1331,2.4818,2.0350,1.7340,2.6863,5.5842,1.1332,4.6055,2.2437,3.8154,1.6393,1.9209,0.7984,1.5175,0.5605,1.5492,1.8893,2.0080,4.8112,2.2913,0.2019,1.2415,0.6952,0.3134,0.5556,2.7144,0.1014,0.4320,2.9859,1.5970,3.4367,2.0991,3.8527,0.4848,2.6743,0.7425,2.6301,0.3077,2.1161